In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

tff.backends.reference.set_reference_context()

In [3]:
from tensorflow.keras import layers
from tensorflow.keras import activations

import math
import pickle
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

import seaborn as sns
sns.set_style("whitegrid")

In [4]:
origTra = open('../Datasets/pendigits/pendigits-orig.tra', 'r').read()

In [5]:
listOrigTra = []
for i in origTra.split('COMMENT ')[1:]:
    listOrigTra.append(i.split()[1])
    
len(listOrigTra)

7494

In [6]:
tra = open('../Datasets/pendigits/pendigits.tra', 'r').read()
listTra = []
for i in tra.split('\n'):
    listTra.append(i[-1])
    
len(listTra)

7494

In [7]:
origTes = open('../Datasets/pendigits/pendigits-orig.tes', 'r').read()

In [8]:
listOrigTes = []
for i in origTes.split('COMMENT ')[1:]:
    listOrigTes.append(i.split()[1])
    
len(listOrigTes)

3498

In [9]:
listOrigTes = [str(int(i)+30) for i in listOrigTes]

In [10]:
tes = open('../Datasets/pendigits/pendigits.tes', 'r').read()
listTes = []
for i in tes.split('\n'):
    listTes.append(i[-1])
    
len(listTes)

3498

In [11]:
columns = ['Input1', 'Input2', 'Input3', 'Input4', 'Input5', 'Input6', 'Input7', 'Input8', 'Input9', 'Input10', 'Input11', 'Input12', 'Input13', 'Input14', 'Input15', 'Input16', 'digit']

In [12]:
df_tra = pd.read_csv('../Datasets/pendigits/pendigits.tra', delimiter = ',', names=columns , header=None)
df_tra.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [13]:
df_tra_w = df_tra.copy()
df_tra_w['writer'] = listOrigTra
df_tra_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8,12
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2,26
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1,11
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4,14
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1,10


In [14]:
df_tes = pd.read_csv('../Datasets/pendigits/pendigits.tes', delimiter = ',', names=columns , header=None)
df_tes.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9


In [15]:
df_tes_w = df_tes.copy()
df_tes_w['writer'] = listOrigTes
df_tes_w.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit,writer
0,88,92,2,99,16,66,94,37,70,0,0,24,42,65,100,100,8,36
1,80,100,18,98,60,66,100,29,42,0,0,23,42,61,56,98,8,42
2,0,94,9,57,20,19,7,0,20,36,70,68,100,100,18,92,8,35
3,95,82,71,100,27,77,77,73,100,80,93,42,56,13,0,0,9,37
4,68,100,6,88,47,75,87,82,85,56,100,29,75,6,0,0,9,35


In [16]:
df = pd.concat([df_tra,df_tes])
df.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16,digit
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98,8
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6,2
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0,1
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0,4
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0,1


In [29]:
labels = df['digit'].to_numpy()
df = df.drop('digit', axis=1)
df.head()

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16
0,47,100,27,81,57,37,26,0,0,23,56,53,100,90,40,98
1,0,89,27,100,42,75,29,45,15,15,37,0,69,2,100,6
2,0,57,31,68,72,90,100,100,76,75,50,51,28,25,16,0
3,0,100,7,92,5,68,19,45,86,34,100,45,74,23,67,0
4,0,67,49,83,100,100,81,80,60,60,40,40,33,20,47,0


In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(df,  labels, test_size = 0.20, random_state = 0)

In [47]:
X_train

,Input1,Input2,Input3,Input4,Input5,Input6,Input7,Input8,Input9,Input10,Input11,Input12,Input13,Input14,Input15,Input16
3270,0,84,37,100,75,85,75,43,67,0,47,5,56,24,100,28
6384,0,45,36,69,68,93,88,100,84,75,84,50,84,24,100,0
1667,23,88,0,55,25,50,64,20,17,0,21,54,35,97,100,100
2578,63,100,31,66,0,31,28,22,94,30,100,68,82,38,81,0
5566,11,77,84,63,79,100,0,83,56,67,100,56,87,7,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4859,13,69,78,100,100,73,59,36,11,0,87,11,87,45,0,69
3264,0,100,15,95,1,63,33,40,100,48,94,65,93,32,90,0
2351,99,74,62,100,23,75,60,53,95,58,100,11,49,1,0,0
3305,11,90,0,41,31,0,82,0,100,46,77,92,30,100,3,58


In [32]:
X_train.shape

(8793, 16)

In [33]:
X_test.shape

(2199, 16)

In [34]:
Y_train.shape

(8793,)

In [35]:
Y_test.shape

(2199,)

In [39]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(X_train.shape[0], activation='relu'),
  tf.keras.layers.Dense(10,activation="sigmoid")
])

In [41]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [42]:
model.fit(X_train.to_numpy(), Y_train, epochs=5)

Epoch 1/5
275/275 [==============================] - 1s 3ms/step - loss: 3.2788 - accuracy: 0.7993
Epoch 2/5
275/275 [==============================] - 1s 3ms/step - loss: 0.3230 - accuracy: 0.9351
Epoch 3/5
275/275 [==============================] - 1s 3ms/step - loss: 0.1056 - accuracy: 0.9714
Epoch 4/5
275/275 [==============================] - 1s 3ms/step - loss: 0.1060 - accuracy: 0.9726
Epoch 5/5
275/275 [==============================] - 1s 3ms/step - loss: 0.1240 - accuracy: 0.9662


In [43]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8793)              149481    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                87940     
Total params: 237,421
Trainable params: 237,421
Non-trainable params: 0
_________________________________________________________________


In [44]:
score = model.evaluate(np.array(X_test), np.array(Y_test), verbose = 0) 

print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

Test loss: 0.19637662172317505
Test accuracy: 0.9740791320800781


In [46]:
for layer in model.layers:
    print(layer.get_config(), layer.get_weights())

{'name': 'dense_3', 'trainable': True, 'dtype': 'float32', 'units': 8793, 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None} [array([[-0.04730536, -0.02600369, -0.01529138, ..., -0.02610187,
         0.01006912, -0.01462855],
       [ 0.02616118, -0.01479198, -0.01400617, ...,  0.01398992,
         0.00778339, -0.01440656],
       [ 0.01518582, -0.00146202, -0.02498533, ..., -0.02173379,
        -0.00399829, -0.00047395],
       ...,
       [-0.01538811,  0.0155643 ,  0.02352145, ..., -0.00946389,
         0.01279589, -0.02737818],
       [ 0.02493115, -0.05999047, -0.04350709, ..., -0.03081804,
        -0.03212812,  0.01665128],
       [-0.00508401, -0.00119956,  0.01618043, ..., -0.00173621,
        -0.00040888,  0.02441407